In [1]:
import pandas as pd
import numpy as np
import os

# Lecture 4 – Messy Data

## DSC 80, Spring 2022

### Announcements

- Lab 1 is due **tonight at 11:59PM**.
    - Watch [this video 🎥](https://youtu.be/FpTo4AM9B30) for setup instructions.
    - Please submit the [Welcome + Alternate Exams Form](https://docs.google.com/forms/d/e/1FAIpQLSdBKLcPs4Xi0plaIw0MVZ0DyGcvnSZyHxKVC7S7LwEiCchepQ/viewform) **by tonight**.
- Project 1 is released!
    - Watch [this video 🎥](https://www.youtube.com/watch?v=Os-BT0FTzVg) to get an overview of the project, and use [this sheet](https://docs.google.com/spreadsheets/d/1PMtGpd4U6rYBn6Ut6eHQzSo4PdBwluU-ppx87ROy_N8/edit#gid=0) to find a pair programming partner.
    - The Checkpoint is due on **Thursday, April 7th at 11:59PM**.
    - The whole project is due on **Thursday, April 14th at 11:59PM**.

### Agenda

- Recap: adding columns.
- Introduction to messy data and data cleaning.
- Kinds of variables.
- Unfaithful data.

## Recap: adding columns

In [2]:
elections_fp = os.path.join('data', 'elections.csv')
elections = pd.read_csv(elections_fp)
elections.head(10)

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss
5,Bush,Republican,53.4,1988,win
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
8,Bush,Republican,37.4,1992,loss
9,Perot,Independent,18.9,1992,loss


### Adding and modifying columns, using a copy

* To add a new column to a DataFrame, use the `assign` method.
* To add a new row to a DataFrame, use the `append` method.
* Both `assign` and `append` return a copy of the DataFrame, **which is a great feature!**
* To change the values in a column, re-assign its name to a sequence of the desired values.

As an aside, you should try your best to write **chained** `pandas` code, as follows:

In [4]:
elections['%'] / 100

0     0.507
1     0.410
2     0.066
3     0.588
4     0.376
5     0.534
6     0.456
7     0.430
8     0.374
9     0.189
10    0.492
11    0.407
12    0.084
13    0.484
14    0.479
15    0.483
16    0.507
17    0.529
18    0.457
19    0.511
20    0.472
21    0.482
22    0.461
23    0.513
24    0.469
Name: %, dtype: float64

In [9]:
elections = elections.assign(prop_of_vote = elections['%']/100)

In [10]:
elections

,Candidate,Party,%,Year,Result,prop_of_vote
0,Reagan,Republican,50.7,1980,win,0.507
1,Carter,Democratic,41.0,1980,loss,0.410
2,Anderson,Independent,6.6,1980,loss,0.066
3,Reagan,Republican,58.8,1984,win,0.588
4,Mondale,Democratic,37.6,1984,loss,0.376
5,Bush,Republican,53.4,1988,win,0.534
6,Dukakis,Democratic,45.6,1988,loss,0.456
7,Clinton,Democratic,43.0,1992,win,0.430
8,Bush,Republican,37.4,1992,loss,0.374
9,Perot,Independent,18.9,1992,loss,0.189


In [11]:
(
    elections
    .assign(proportion_of_vote=(elections['%'] / 100))
    .head()
)

,Candidate,Party,%,Year,Result,prop_of_vote,proportion_of_vote
0,Reagan,Republican,50.7,1980,win,0.507,0.507
1,Carter,Democratic,41.0,1980,loss,0.410,0.410
2,Anderson,Independent,6.6,1980,loss,0.066,0.066
3,Reagan,Republican,58.8,1984,win,0.588,0.588
4,Mondale,Democratic,37.6,1984,loss,0.376,0.376


You can chain together several steps at a time:

In [12]:
(
    elections
    .assign(proportion_of_vote=(elections['%'] / 100))
    .assign(Result=elections['Result'].str.upper())
    .head()
)

,Candidate,Party,%,Year,Result,prop_of_vote,proportion_of_vote
0,Reagan,Republican,50.7,1980,WIN,0.507,0.507
1,Carter,Democratic,41.0,1980,LOSS,0.410,0.410
2,Anderson,Independent,6.6,1980,LOSS,0.066,0.066
3,Reagan,Republican,58.8,1984,WIN,0.588,0.588
4,Mondale,Democratic,37.6,1984,LOSS,0.376,0.376


You can also use `assign` when the desired column name has spaces, by using keyword arguments.

In [13]:
(
    elections
    .assign(**{'Proportion of Vote': elections['%'] / 100})
    .head()
)

,Candidate,Party,%,Year,Result,prop_of_vote,Proportion of Vote
0,Reagan,Republican,50.7,1980,win,0.507,0.507
1,Carter,Democratic,41.0,1980,loss,0.410,0.410
2,Anderson,Independent,6.6,1980,loss,0.066,0.066
3,Reagan,Republican,58.8,1984,win,0.588,0.588
4,Mondale,Democratic,37.6,1984,loss,0.376,0.376


### Adding and modifying columns, in-place

* You can assign a new row or column to a DataFrame **in-place** using `loc` or `[]`.
    - Works like dictionary assignment.
    - Unlike `assign`, this **modifies** the underlying DataFrame rather than a copy of it.
* This is the more "common" way of adding/modifying columns. 
    - ⚠️ Warning: Exercise caution when using this approach, since this approach changes the values of existing variables.

In [14]:
# By default, .copy() returns a deep copy of the object it is called on,
# meaning that if you change the copy the original remains unmodified.
mod_elec = elections.copy()
mod_elec.head()

,Candidate,Party,%,Year,Result,prop_of_vote
0,Reagan,Republican,50.7,1980,win,0.507
1,Carter,Democratic,41.0,1980,loss,0.410
2,Anderson,Independent,6.6,1980,loss,0.066
3,Reagan,Republican,58.8,1984,win,0.588
4,Mondale,Democratic,37.6,1984,loss,0.376


In [15]:
mod_elec['Proportion of Vote'] = mod_elec['%'] / 100
mod_elec.head()

,Candidate,Party,%,Year,Result,prop_of_vote,Proportion of Vote
0,Reagan,Republican,50.7,1980,win,0.507,0.507
1,Carter,Democratic,41.0,1980,loss,0.410,0.410
2,Anderson,Independent,6.6,1980,loss,0.066,0.066
3,Reagan,Republican,58.8,1984,win,0.588,0.588
4,Mondale,Democratic,37.6,1984,loss,0.376,0.376


In [16]:
mod_elec['Result'] = mod_elec['Result'].str.upper()
mod_elec.head()

,Candidate,Party,%,Year,Result,prop_of_vote,Proportion of Vote
0,Reagan,Republican,50.7,1980,WIN,0.507,0.507
1,Carter,Democratic,41.0,1980,LOSS,0.410,0.410
2,Anderson,Independent,6.6,1980,LOSS,0.066,0.066
3,Reagan,Republican,58.8,1984,WIN,0.588,0.588
4,Mondale,Democratic,37.6,1984,LOSS,0.376,0.376


In [17]:
# 🤔
mod_elec.loc[-1, :] = ['Carter', 'Democratic', 50.1, 1976, 'WIN', 0.501]
mod_elec.loc[-2, :] = ['Ford', 'Republican', 48.0, 1976, 'LOSS', 0.48]
mod_elec

ValueError: Must have equal len keys and value when setting with an iterable

In [18]:
mod_elec = mod_elec.sort_index()
mod_elec.head()

,Candidate,Party,%,Year,Result,prop_of_vote,Proportion of Vote
-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Reagan,Republican,50.7,1980.0,WIN,0.507,0.507
1,Carter,Democratic,41.0,1980.0,LOSS,0.410,0.410
2,Anderson,Independent,6.6,1980.0,LOSS,0.066,0.066
3,Reagan,Republican,58.8,1984.0,WIN,0.588,0.588


In [19]:
# df.reset_index(drop=True) drops the current index 
# of the DataFrame and replaces it with an index of increasing integers
mod_elec.reset_index(drop=True)

,Candidate,Party,%,Year,Result,prop_of_vote,Proportion of Vote
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Reagan,Republican,50.7,1980.0,WIN,0.507,0.507
2,Carter,Democratic,41.0,1980.0,LOSS,0.410,0.410
3,Anderson,Independent,6.6,1980.0,LOSS,0.066,0.066
4,Reagan,Republican,58.8,1984.0,WIN,0.588,0.588
5,Mondale,Democratic,37.6,1984.0,LOSS,0.376,0.376
6,Bush,Republican,53.4,1988.0,WIN,0.534,0.534
7,Dukakis,Democratic,45.6,1988.0,LOSS,0.456,0.456
8,Clinton,Democratic,43.0,1992.0,WIN,0.430,0.430
9,Bush,Republican,37.4,1992.0,LOSS,0.374,0.374


## Introduction to messy data

### There is no such thing as "raw data"!

* Data are the result of measurements that must be recorded.
* Humans design the measurements and record the results.
* Data is **always** an imperfect record of the underlying processing being measured.

### Data generating process

* A **data generating process** is the underlying, real-world (probabilistic) mechanism that generates observed data. 
* Observed data is an incomplete artifact of the data generating process.
* **A data generating process is what a statistical model attempts to describe.**
    - From DSC 10: a model is a set of assumptions about how data were generated.
    - More on this later in the quarter.
- Data cleaning requires an understanding of the data generating process.

### Example: COVID case counts 🦠

Suppose our **goal** is to determine the number of COVID cases in the US **yesterday**.
- What are we really asking for – the number of people who tested positive yesterday, or the number of people who contracted COVID yesterday?
- Tested positive on what type of test? How accurate is that type of test?
- How often are test results reported? Is there a delay in when test results are reported?

<center><img src='imgs/christmas.png' width=70%></center>

Why do you think so few cases were reported on Christmas Day – is it because COVID was less prevalent on Christmas Day as compared to the days before and after, or is it likely for some other reason? 🎅

### Data provenance

- As data scientists, we often need to work with datasets that others collected, for a purpose that is different than our current interest.
- As such, it's important to understand the "story" of how a dataset came to be, or the **provenance** of the data. Specifically, we need to be aware of:
    1. Assumptions about the data generating process.
    2. How the initial values in the dataset came to be.  
    3. How any data processing or storage decisions affected the values in the dataset.

The bigger picture question we're asking here is, **can we trust our data?**

### Data cleaning 🧹

- Data cleaning is the process of transforming data so that it best represents the underlying data generating process.

- In practice, data cleaning is often detective work to understand data provenance.
    - **Always be skeptical of your data!**

### Keys to data cleaning

Data cleaning often addresses: 

* The **structure** of the recorded data.
    - Is the data stored in a tabular format (e.g. CSV, SQL, Google Sheets) or in another format (JSON, XML)?
    - Are the individuals properly represented as rows?
* The **encoding** and **format** of the values in the data.
    - Are the data types of all columns reflective of the **kinds of data** they contain?
* Corrupt and "**incorrect**" data, and missing values.
    - Were there flaws in the data recording process? In other words, is our data **faithful** to the data generating process?
    
Let's focus on the latter two.

## Kinds of data

### Kinds of data

<center><img src='imgs/data-types.png' width=90%></center>

### Discussion Question

Determine the kind of each of the following variables.
- Fuel economy in miles per gallon.
- Number of quarters at UCSD.
- Class standing (freshman, sophomore, etc.).
- Income bracket (low, medium, high).
- Bank account number.

### Example: DSC 80 students

In the next cell, we'll load in an example dataset containing information about past DSC 80 students.

- `'PID'` and `'Student Name'`: student PID and name.
- `'Month'`, `'Day'`, `'Year'`: date when the student was accepted to UCSD.
- `'2021 tuition'` and `'2022 tuition'`: amount paid in tuition in 2021 and 2022, respectively.
- `'Percent Growth'`: growth between the two aforementioned columns.
- `'Paid'`: whether or not the student has paid tuition for this quarter yet.
- `'DSC 80 Final Grade'`: either `'Pass'`, `'Fail'`, or a number.

What needs to be changed in the DataFrame to compute statistics?

In [40]:
students = pd.read_csv(os.path.join('data', 'students.csv'))
students

,Student ID,Student Name,Month,Day,Year,2021 tuition,2022 tuition,Percent Growth,Paid,DSC 80 Final Grade
0,A20104523,John Black,10,12,2020,$40000.00,$50000.00,25.00%,N,89
1,A20345992,Mark White,4,15,2019,$9200.00,$10120.00,10.00%,Y,90
2,A21942188,Amy Red,5,14,2021,$50000.00,$62500.00,25.00%,N,97
3,A28049910,Tom Green,7,10,2020,$7000.00,$9800.00,40.00%,Y,54
4,A27456704,Rose Pink,3,3,2021,$10000.00,$5000.00,-50.00%,Y,Pass


### How much has each student paid in total tuition in 2021 and 2022?

In [21]:
students

,Student ID,Student Name,Month,Day,Year,2021 tuition,2022 tuition,Percent Growth,Paid,DSC 80 Final Grade
0,A20104523,John Black,10,12,2020,$40000.00,$50000.00,25.00%,N,89
1,A20345992,Mark White,4,15,2019,$9200.00,$10120.00,10.00%,Y,90
2,A21942188,Amy Red,5,14,2021,$50000.00,$62500.00,25.00%,N,97
3,A28049910,Tom Green,7,10,2020,$7000.00,$9800.00,40.00%,Y,54
4,A27456704,Rose Pink,3,3,2021,$10000.00,$5000.00,-50.00%,Y,Pass


In [30]:
total = students['2021 tuition'] + students['2022 tuition']
total

0     90000.0
1     19320.0
2    112500.0
3     16800.0
4     15000.0
dtype: float64

### Check the data types of `students`!

* What kinds of data should each column have?
    - Qualitative or quantitative?
    - Discrete or continuous?
    - Ordinal or nominal?
* What data type *should* each column have?

* Use the `dtypes` attribute (or the `info` method) to peek at the data types.

In [32]:
students.dtypes

Student ID             object
Student Name           object
Month                   int64
Day                     int64
Year                    int64
2021 tuition          float64
2022 tuition          float64
Percent Growth         object
Paid                   object
DSC 80 Final Grade     object
dtype: object

### Cleaning `'2021 tuition'` and `'2022 tuition'`

* `'2021 tuition'` and `'2022 tuition'` are stored as `object`s (strings), not numerical values.
* The `'$'` character causes the entries to be interpreted as strings.
* We can use `str` methods to strip the dollar sign.

In [34]:
# This won't work. Why?
students['2021 tuition'].astype(float)

ValueError: could not convert string to float: '$40000.00'

In [35]:
# That's better!
students['2021 tuition'].str.strip('$').astype(float)

0    40000.0
1     9200.0
2    50000.0
3     7000.0
4    10000.0
Name: 2021 tuition, dtype: float64

We can loop through the columns of `students` to apply the above procedure. (Looping through columns is fine, just avoid looping through rows.)

In [25]:
for col in students.columns:
    if 'tuition' in col:
        students[col] = students[col].str.strip('$').astype(float)
        
students

,Student ID,Student Name,Month,Day,Year,2021 tuition,2022 tuition,Percent Growth,Paid,DSC 80 Final Grade
0,A20104523,John Black,10,12,2020,40000.0,50000.0,25.00%,N,89
1,A20345992,Mark White,4,15,2019,9200.0,10120.0,10.00%,Y,90
2,A21942188,Amy Red,5,14,2021,50000.0,62500.0,25.00%,N,97
3,A28049910,Tom Green,7,10,2020,7000.0,9800.0,40.00%,Y,54
4,A27456704,Rose Pink,3,3,2021,10000.0,5000.0,-50.00%,Y,Pass


Alternatively, we can do this without a loop by using `str.contains` to find only the columns that contain tuition information.

In [38]:
cols = students.columns.str.contains('tuition')
students.loc[:, cols] = students.loc[:, cols].astype(float)
students

ValueError: could not convert string to float: '$40000.00'

### Cleaning `'Paid'`

* Currently, `'Paid'` contains the strings `'Y'` and `'N'`.
    * `'Y'`s and `'N'`s typically result from manual data entry.
* The `'Paid'` column should contain `True`s and `False`s, or `1`s and `0`s.
* Solutions:
    - Use the `replace` Series method.
    - Create a Boolean Series through comparison.

In [36]:
students['Paid'].replace({'Y': True, 'N': False})

0    False
1     True
2    False
3     True
4     True
Name: Paid, dtype: bool

In [41]:
students['Paid'].value_counts()

Y    3
N    2
Name: Paid, dtype: int64

In [42]:
students['Paid'] = students['Paid'] == 'Y'
students

,Student ID,Student Name,Month,Day,Year,2021 tuition,2022 tuition,Percent Growth,Paid,DSC 80 Final Grade
0,A20104523,John Black,10,12,2020,$40000.00,$50000.00,25.00%,False,89
1,A20345992,Mark White,4,15,2019,$9200.00,$10120.00,10.00%,True,90
2,A21942188,Amy Red,5,14,2021,$50000.00,$62500.00,25.00%,False,97
3,A28049910,Tom Green,7,10,2020,$7000.00,$9800.00,40.00%,True,54
4,A27456704,Rose Pink,3,3,2021,$10000.00,$5000.00,-50.00%,True,Pass


### Cleaning `'Month'`, `'Day'`, and `'Year'`
* Currently, these are stored separately using the `int64` data type. This could be *fine* for certain purposes, but ideally they are stored as a single column (e.g. for sorting).
* Solutions:
    * Store dates as strings of the form `'YYYY-MM-DD'`.
    * Store dates as `datetime64` objects (later).

In [ ]:
(
    students['Year'].astype(str) + '-' + 
    students['Month'].astype(str).str.zfill(2) + '-' + 
    students['Day'].astype(str).str.zfill(2)
)

Note:
- Due to **broadcasting**, we were able to add a Series to a string.
- The `zfill` string method adds zeroes to the start of a string until it reaches the specified length.

### Cleaning `'DSC 80 Final Grade'`

* Currently, `'DSC 80 Final Grade'`s are stored as `object`s (strings).
* Unless we somehow store this column to a numeric type, we can't do any arithmetic with it.
* However, due to the existence of strings like `'Pass'`, we can't use `astype` to convert it.
* Solution: use `pd.to_numeric(s, errors='coerce')`, where `s` is a Series.
    - ⚠️ Be careful with this!
    - `errors='coerce'` can cause uninformed destruction of data.

In [ ]:
# Won't work!
students['DSC 80 Final Grade'].astype(int)

In [43]:
pd.to_numeric(students['DSC 80 Final Grade'], errors='coerce')

0    89.0
1    90.0
2    97.0
3    54.0
4     NaN
Name: DSC 80 Final Grade, dtype: float64

In [44]:
students['DSC 80 Final Grade'] = pd.to_numeric(students['DSC 80 Final Grade'], errors='coerce')
students

,Student ID,Student Name,Month,Day,Year,2021 tuition,2022 tuition,Percent Growth,Paid,DSC 80 Final Grade
0,A20104523,John Black,10,12,2020,$40000.00,$50000.00,25.00%,False,89.0
1,A20345992,Mark White,4,15,2019,$9200.00,$10120.00,10.00%,True,90.0
2,A21942188,Amy Red,5,14,2021,$50000.00,$62500.00,25.00%,False,97.0
3,A28049910,Tom Green,7,10,2020,$7000.00,$9800.00,40.00%,True,54.0
4,A27456704,Rose Pink,3,3,2021,$10000.00,$5000.00,-50.00%,True,NaN


In [ ]:
pd.to_numeric?

### Cleaning `'Student Name'`
* We want names to be formatted as `'Last Name, First Name'`, a common format.
* One solution: use the Series `apply` method.
    - If `s` is a Series, `s.apply(func)` applies the function `func` to each entry of `s`.

In [ ]:
students['Student Name']

In [ ]:
def transpose_name(name):
    firstname, lastname = name.split()
    return lastname + ', ' + firstname

transpose_name('King Triton')

In [ ]:
students['Student Name'].apply(transpose_name)

### Aside: string methods

`str` methods are useful – use them!
- To use them, access the `str` attribute of Series.
- Then, whatever method/operator comes immediately after will be applied to each element of the Series individually, rather than the Series as a whole.

In [45]:
parts = students['Student Name'].str.split()
parts

0    [John, Black]
1    [Mark, White]
2       [Amy, Red]
3     [Tom, Green]
4     [Rose, Pink]
Name: Student Name, dtype: object

In [46]:
parts.str[1] + ', ' + parts.str[0]

0    Black, John
1    White, Mark
2       Red, Amy
3     Green, Tom
4     Pink, Rose
Name: Student Name, dtype: object

### More data type ambiguities

- 1649043031 looks like a number, but is probably a date.
    - [Unix timestamps](https://www.unixtimestamp.com) count the number of seconds since January 1st, 1970.

- "USD 1,000,000" looks like a string, but is actually a number **and** a unit.
    
- 92093 looks like a number, but is really a zip code (and isn't equal to 92,093).
    
- Sometimes, `False` appears in a column of country codes. Why might this be? 
🤔

### Example: the Norway problem 🇳🇴

In [ ]:
import yaml

player = '''
name: Magnus Carlsen
age: 31
country: NO
'''

In [ ]:
yaml.safe_load(player)

## Unfaithful data

### Is the data "faithful" to the DGP?

- In other words, how well does the data represent reality?

- Does the data contain unrealistic or "incorrect" values?
    - Dates in the future for events in the past.
    - Locations that don't exist.
    - Negative counts.
    - Misspellings of names.
    - Large outliers.

### Is the data "faithful" to the DGP?
    
- Does the data violate obvious dependencies?
    - Age and birthday don't match. 
- Was the data entered by hand?
     - Spelling errors.
     - Fields shifted.
     - Did the form require fields or provide default values?  
- Are there obvious signs of data falsification (aka "curbstoning")?
    - Repeated names.
    - Fake looking email addresses.
    - Repeated use of uncommon names or fields.

<center><img src='imgs/data-sd.png' width=70%></center>

### Example: Police vehicle stops 🚔

The dataset we're working with contains all of the vehicle stops that the San Diego Police Department made in 2016.

<center><img src="imgs/image_5.png"/></center>

### General questions

1. Check the data types. Notice any issues?
2. Do string fields have consistent values?
3. Are there missing values that we don't understand?
4. Are all values within a reasonable range?
5. How do we deal with the messiness we find?

In [ ]:
stops = pd.read_csv('data/vehicle_stops_2016_datasd.csv')
stops.head()

### Data types
* Are the data types correct?
* If not, are they easily fixable?

In [ ]:
stops.head(1)

In [ ]:
stops.info()

### Unfaithfulness
* Are there suspicious values?
* If a value is suspicious, can we trust the observation?
* For example, consider `'subject_age'` – some are too old to be true, some are too low to be true.

In [ ]:
stops['subject_age'].unique()

In [ ]:
ages = pd.to_numeric(stops['subject_age'], errors='coerce')
ages.describe()

Ages range all over the place, from 0 to 220. Was a 220 year old really pulled over?

In [ ]:
stops.loc[ages > 100]

In [ ]:
ages.loc[(ages >= 0) & (ages < 16)].value_counts()

In [ ]:
stops.loc[(ages >= 0) & (ages < 16)]

### Unfaithful `'subject_age'`

* Ages of `'No Age'` and `0` are likely explicit null values.
* What do we do about the exceptionally small and large ages?
    - Do we throw the entire row away, even if the rest of row is well-formed?
* What about the 14 and 15 year olds?
    - Each has more than one occurrence – these could be real entries!

In the coming weeks, we'll cover more solutions to these problems.

## Summary, next time

### Summary

- Data provenance describes the "origin story" of a dataset, from the data generating process to its storage.
- Data cleaning is the process of transforming data so that it best represents the underlying data generating process.
- We must ensure that each column in a DataFrame uses the correct data type for the **kind** of data in the column.
- We must also ensure that our data is **faithful** to the data generating process, by looking for missing or strange values.
- **Next time:** finish discussing unfaithful data, and (re)introduce hypothesis testing.